In [11]:
import pandas as pd
import numpy as np
import sys
import os
import scipy.stats as stats
from statistics import mean
from enum import Enum
import sys
print(sys.version)

class DB(Enum):
    Mcyt=0
    eBioSignDS1=1
    eBioSignDS2=3
    BiosecurID=4
    BiosecureDS2=5
    EvalDB=6

class Signature:
    def __init__(self, data, filePath):
        self.data = data
        self.filePath = filePath
        pathParts= filePath.split('/')
        self.inputDevice=pathParts[2]
        if "signature" in filePath:
            self.db='EvalDb'
            self.inputDevice='Unknown'
            return
        
        parts=pathParts[4].split('_')
        signerId=parts[0][1:]
        self.db=self.getDatabase(pathParts[2],pathParts[3],int(signerId))
        print(self.db)
        pressureColumn=self.getPressureColumn()
        
        self.x=data[:, 0].tolist()
        self.y=data[:, 1].tolist()
        self.t=data[:, 2].tolist()
        self.pressure=data[:, pressureColumn]
        
        if (self.inputDevice == "Unkown"):
            self.inputDevice=getInputDeviceForEvalDb()
        
        self.correctZeroTime()
        self.correctSameTimestamps()
    
        if self.inputDevice == 'stylus':
            i=0
            while len(self.pressure) > 0 and self.pressure[0] == 0:
                self.x.pop(0)
                self.y.pop(0)
                self.t.pop(0)
                self.pressure.pop(0)

            while (len(self.pressure) > 0 and self.pressure[len(self.pressure)-1] == 0):
                self.x.pop(len(self.pressure)-1)
                self.y.pop(len(self.pressure)-1)
                self.t.pop(len(self.pressure)-1)
                self.pressure.pop(len(self.pressure)-1)
        
        self.dx=difference(self.x)
        #self.dx=self.x.diff()
        for dx in self.dx:
            print(dx)
    
    def getPressureColumn(self):
        if self.db==DB.Mcyt:
            return 5
        elif self.db==DB.BiosecurID:
            return 6
        elif self.db==DB.BiosecureDS2:
            return 6
        elif self.db==DB.eBioSignDS1:
            return 3
        elif self.db==DB.eBioSignDS2:
            return 3
        elif self.db==DB.EvalDB:
            return 3
        
    def getDatabase(self, split, inputDevice, signerId):
        '''
        print(split)
        print(inputDevice)
        print(signerId)
        '''
        if split=='Development':
            if inputDevice=='finger':
                if 1009<=signerId<=1038:
                    return DB.eBioSignDS1
                elif 1039<=signerId<=1084:
                    return DB.eBioSignDS2
                else:
                    print("Undefined DB for file: {file}")
            elif inputDevice=="stylus":
                if 1<=signerId<=230: 
                    return DB.Mcyt
                elif 231<=signerId<=498:
                    return DB.BiosecurID
                elif 1009<=signerId<=1038:
                    return DB.eBioSignDS1
                elif 1039<=signerId<=1084:
                    return DB.eBioSignDS2
                else:
                    print("Undefined DB for file: {file}")
            else:
                print("Undefined InputDevice for file: {file}")
        elif split=='Evaluation':
            if inputDevice=='finger':
                if 373<=signerId<=407:
                    return DB.eBioSignDS1
                elif 408<=signerId<=442:
                    return DB.eBioSignDS2
                else:
                    print("Undefined DB for file: {file}")
            elif inputDevice=="stylus":
                if 1<=signerId<=100:
                    return DB.Mcyt
                elif 101<=signerId<=232:
                    return DB.BiosecurID
                elif 233<=signerId<=372:
                    return DB.BiosecureDS2
                elif 373<=signerId<=407:
                    return DB.eBioSignDS1
                elif 408<=signerId<=442:
                    return DB.eBioSignDS2
                else:
                    print("Undefined DB for file: {file}")
            else:
                print("Undefined InputDevice for file: {file}")
        else:
            print("Undefined InputDevice for file: {file}")
            
    def getInputDeviceForEvalDb(self):
        for i in range(0, len(self.pressure), 1):
            if self.pressure[i]!=0:
                return False

        return True
    
    def isAllZero(self):
        for i in range(0, self.data.shape[0], 1):
            if self.data[i,0]!=0:
                return False

        return True
    
    def correctZeroTime(self):
        if self.isAllZero():
            for i in range(0, self.data.shape[0], 1):
                self.data[i,0]=i*10
                
    def correctSameTimestamps(self):
        i=0
        while i < self.data.shape[0]-1:
            if self.data[i,0]==self.data[i+1,0]:
                self.data=np.delete(self.data, i, 0)
            else:
                i += 1

def readFilesInDir(dirPath, allFiles):
    signatureFiles = os.listdir(dirPath)
    for file in signatureFiles:
        dataframe=pd.read_csv(dirPath + '/' + file,sep=' ',skiprows=[0],header=None)
        array=np.array(dataframe)
        signature=Signature(array,dirPath + '/' + file)
        allFiles.append(signature)
        
def difference(dataset, interval=1):
    diff = []
    for i in range(interval, len(dataset)):
        value = dataset[i] - dataset[i - interval]
        diff.append(value)
        
    return diff

def Average(lst):
    return sum(lst) / len(lst)

def normalizeX(allFiles):
    all=[]
    for item in allFiles:
        all=all+item.x
    
    #print('xAll')
    #print(xAll)
    #average=mean(xAll)
    z = stats.zscore(all)
    #print(average)
    print(z)
    allIndex=0
    
    for item in allFiles:
        for i in range(0,len(item.x),1):
            item.x[i]=z[allIndex]
            allIndex+=1
            
    print('Normalized x:')
    for item in allFiles:
        print(item.x)
    '''
    for item in allFiles:
        for i in range(0,len(item.x),1):
            item.x[i]=(item.x[i]-average)/z
    '''
def normalizeY(allFiles):
    all=[]
    for item in allFiles:
        all=all+item.y
    z = stats.zscore(all)
    print(z)
    
    allIndex=0
    for item in allFiles:
        for i in range(0,len(item.y),1):
            item.y[i]=z[allIndex]
            allIndex+=1
            
    print('Normalized y:')
    for item in allFiles:
        print(item.y)
            
def normalizePressure(allFiles):
    all=[]
    for item in allFiles:
        all=all+item.pressure
    z = stats.zscore(all)
    print(z)
    
    allIndex=0
    for item in allFiles:
        for i in range(0,len(item.pressure),1):
            item.pressure[i]=z[allIndex]
            allIndex+=1
            
    print('Normalized pressure:')
    for item in allFiles:
        print(item.pressure)

def normalizeX2(allFiles):
    for item in allFiles:
        z = stats.zscore(item.x)
        print(z)

        for i in range(0,len(item.x),1):
            item.x[i]=z[i]

    print('Normalized x:')
    for item in allFiles:
        print(item.x)
        
def normalizeY2(allFiles):
    for item in allFiles:
        z = stats.zscore(item.y)
        print(z)

        for i in range(0,len(item.y),1):
            item.y[i]=z[i]

    print('Normalized y:')
    for item in allFiles:
        print(item.y)
        
def normalizePressure2(allFiles):
    for item in allFiles:
        z = stats.zscore(item.pressure)
        print(z)

        for i in range(0,len(item.pressure),1):
            item.pressure[i]=z[i]

    print('Normalized pressure:')
    for item in allFiles:
        print(item.pressure)
            
allFiles=[]
        
#readFilesInDir('C:\DeepSignDB/Evaluation/stylus',allFiles)
#readFilesInDir('C:\DeepSignDB/Evaluation/finger',allFiles)
#readFilesInDir('C:\DeepSignDB/Development/stylus',allFiles)

readFilesInDir('D:/DeepSignSmall/Evaluation/stylus',allFiles)
readFilesInDir('D:/DeepSignSmall/Evaluation/finger',allFiles)
readFilesInDir('D:/DeepSignSmall/Development/stylus',allFiles)
readFilesInDir('D:/DeepSignSmall/Development/finger',allFiles)

'''
for file in allFiles:
    #print(file.data)
    #print(file.filePath)
    #print(file.db)
    #print(file.pressure)
'''
    
print('Allfiles:')
#print(allFiles)
print('Loading data finished')

normalizeX2(allFiles)
normalizeY2(allFiles)
normalizePressure2(allFiles)

3.9.7 (default, Sep 16 2021, 16:59:28) [MSC v.1916 64 bit (AMD64)]
DB.Mcyt
113
-23
-66
-95
-122
-193
-241
-192
-126
-81
0
50
81
147
180
203
204
149
112
30
0
-17
-76
-124
-155
-112
5
84
142
187
217
185
121
78
48
-29
-81
-153
-212
-252
-270
-145
-62
157
262
331
299
227
180
143
49
-13
-106
-217
-291
-362
-398
-421
-158
55
197
356
407
440
352
267
209
43
-21
-63
-155
-281
-364
-398
-362
-338
-28
182
321
423
364
324
181
37
-58
-183
-337
-439
-520
-482
-457
-200
-44
60
119
DB.Mcyt
50
0
-34
-86
-123
-143
-175
-197
-151
-88
-46
14
83
129
175
202
221
210
136
86
12
-32
-61
-207
-274
-319
-244
-83
25
181
285
353
321
279
205
169
-13
-119
-189
-245
-324
-376
-217
-30
94
263
350
409
369
299
252
82
-32
-109
-251
-366
-443
-483
-334
-236
18
193
310
460
432
412
319
171
73
-29
-116
-175
-352
-422
-468
-447
-216
-61
128
296
409
420
384
361
169
47
-34
-118
-304
-427
-484
-506
-522
-413
-227
-104
11
63
98
DB.BiosecureDS2
0
88
54
71
79
71
56
33
0
-32
-90
-147
-207
-246
-254
-230
-165
-77
34
145
260
353
408
4

DB.BiosecureDS2
0
-79
-47
-56
-60
-55
-48
-31
-9
20
61
102
150
193
223
234
215
164
92
0
-82
-176
-261
-326
-365
-359
-298
-195
-53
112
277
445
593
679
702
0
1162
341
133
-78
-270
-422
-520
-568
-567
-522
-453
-375
-291
-209
-133
-58
0
47
0
112
0
0
0
295
143
184
238
284
320
329
306
255
170
75
-28
-135
-228
-307
-364
-379
-351
-285
-179
-62
66
205
345
485
582
617
576
0
770
120
-64
-235
-368
-461
-510
-509
-465
-387
-284
-180
-83
0
0
180
137
154
160
163
160
152
135
108
72
36
0
-18
-50
-81
-110
-130
-132
-112
-76
-32
17
61
101
126
126
100
57
13
-33
-71
-107
-136
-147
-132
-100
-54
0
58
133
220
308
368
387
364
298
212
117
24
-72
-166
-253
-322
-349
-338
-295
-226
-147
-65
16
86
140
160
153
122
75
35
0
-36
-67
-92
-98
-80
-49
-10
24
48
58
65
61
48
34
15
0
DB.BiosecureDS2
0
-41
0
0
0
20
35
63
97
136
165
176
164
125
67
0
-77
-162
-240
-299
-335
-320
-256
-156
-9
157
318
478
593
642
636
553
0
709
100
-73
-238
-362
-439
-471
-465
-438
-388
-332
-271
-205
-143
0
-113
9
41
67
95
126
159
196
238
27

-5
-4
-5
-5
-5
-4
-5
-5
-5
-5
-4
-5
-4
-5
-4
-4
-4
-3
-4
-3
-3
-3
-3
-2
-2
-1
-2
0
-1
0
0
0
0
1
1
1
1
2
2
3
2
3
4
3
4
4
5
5
5
5
5
5
6
6
6
6
6
6
6
6
6
6
6
6
5
5
6
4
5
5
4
4
3
3
3
3
2
2
2
1
1
0
0
0
-1
-1
-2
-3
-5
-5
-6
-6
-6
-6
-6
-7
-6
-7
-7
-6
-7
-7
-7
-7
-8
-7
-7
-7
-8
-7
-7
-6
-6
-6
-5
-3
-4
-2
-1
-1
0
2
2
3
4
5
DB.eBioSignDS1
0
0
0
0
0
0
0
0
0
0
0
0
-7
-10
-9
-4
-6
-16
0
-2
-5
-5
-12
-8
-4
-4
-5
-3
-1
-1
0
0
0
0
0
0
2
5
10
9
15
8
6
10
16
4
5
10
3
4
3
3
5
5
4
3
2
1
1
0
1
0
0
0
0
0
0
0
0
-1
-2
-4
0
-7
-13
-7
-3
-4
-4
-4
-2
-1
-1
-1
0
0
-1
1
1
2
0
1
3
2
2
4
4
9
7
4
2
2
1
1
0
1
0
0
0
0
0
0
0
0
0
-3
0
0
0
0
0
0
0
0
0
2
0
0
8
11
9
5
5
5
10
3
2
1
1
0
0
0
0
0
0
-1
-3
0
0
-3
-6
-8
-7
-9
-6
-4
-3
-2
0
0
0
0
3
1
4
4
2
3
2
5
5
6
6
5
3
1
1
0
0
0
0
0
0
-1
-3
-1
-2
-2
-2
-2
-1
-1
-1
-1
0
0
0
1
1
1
3
2
4
13
16
0
1
0
-1
-4
-7
-6
-4
-11
-17
-13
-12
-26
-13
-17
-5
-17
-15
-8
-19
0
-1
0
0
1
2
5
4
20
7
19
4
28
14
16
5
7
11
6
0
0
0
-3
-5
-8
-7
-6
-7
-15
-61
0
-19
-19
-4
-9
-8
-3
-1
-1
0
3
13
14
22
16
29


30
32
32
32
29
29
28
27
27
25
24
21
20
19
19
19
15
9
5
0
-6
-7
-10
-11
-8
-7
-5
0
0
0
0
0
10
7
13
15
15
20
23
25
26
29
29
31
33
32
30
28
23
17
11
6
0
-5
-8
-10
-10
-6
0
-8
0
0
0
0
7
8
8
11
12
11
13
12
11
11
10
11
11
13
11
11
12
0
19
7
0
6
7
12
16
20
21
19
19
19
17
15
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
5
45
57
68
79
84
87
82
77
68
55
46
31
16
0
-5
-19
-25
-31
-34
-38
-37
-37
-33
-28
-25
-20
-15
-10
-5
0
0
7
7
7
7
5
0
0
0
0
-7
-7
-6
0
-10
-5
0
-10
-6
-7
-10
-12
-16
-21
-25
-29
-29
-28
0
-41
-20
-18
-19
-14
-9
0
0
0
11
16
19
23
27
29
32
32
30
28
27
29
25
24
18
15
13
13
13
11
10
8
6
7
7
6
7
0
5
0
-6
-6
-6
-5
0
-6
0
0
0
5
5
6
9
11
13
15
17
18
17
19
19
18
21
19
19
21
20
17
18
15
11
11
7
5
0
0
0
0
0
0
0
0
0
0
0
9
7
10
9
12
10
9
10
8
8
7
6
6
6
6
6
7
7
7
0
21
7
8
7
10
22
28
34
38
37
36
36
36
35
31
29
24
19
15
12
9
7
5
0
0
0
0
0
0
-19
-10
-12
-16
-20
-24
-26
-28
-31
-29
-30
-30
-26
-26
-23
-16
-8
0
0
9
10
15
20
25
29
33
34
36
37
35
31
26
22
17
11
7
0
-6
-9
-13
-19
-23
-28
-3

6
2
-3
-6
-8
-9
-12
-17
-25
-24
-25
-30
-32
-38
-49
-49
-46
-39
-26
-15
-5
9
32
41
45
56
74
86
104
112
103
101
87
76
60
44
24
-544
-3
0
0
1
5
3
39
-1
0
1
0
1
-210
0
0
3
4
5
5
2
1
15
4
7
11
2
Allfiles:
Loading data finished
[-1.53830340e-01 -1.39117600e-03 -3.24186165e-02 -1.21453881e-01
 -2.49610700e-01 -4.14191037e-01 -6.74551733e-01 -9.99665349e-01
 -1.25867703e+00 -1.42865344e+00 -1.53792399e+00 -1.53792399e+00
 -1.47047303e+00 -1.36120248e+00 -1.16289667e+00 -9.20073219e-01
 -6.46222331e-01 -3.71022424e-01 -1.70018570e-01 -1.89284250e-02
  2.15421496e-02  2.15421496e-02 -1.39117600e-03 -1.03916632e-01
 -2.71195006e-01 -4.80292975e-01 -6.31383120e-01 -6.24638024e-01
 -5.11320416e-01 -3.19759696e-01 -6.74931145e-02  2.25244042e-01
  4.74812585e-01  6.38043902e-01  7.43267396e-01  8.08020315e-01
  7.68898760e-01  6.59628209e-01  4.53228278e-01  1.67236218e-01
 -1.72716608e-01 -5.36951780e-01 -7.32559557e-01 -8.16198744e-01
 -6.04402737e-01 -2.50959719e-01  1.95565620e-01  5.98922347e-

[-1.00075709 -0.78637397 -0.74013447 -0.59721239 -0.52154776 -0.41225441
 -0.3702185  -0.23150001 -0.16844615 -0.0591528   0.08797287  0.18465545
  0.21828418  0.27293086  0.40324216  0.39063139  0.49992475  0.68488273
  0.743733    0.73952941  0.72691864  0.743733    0.73952941  0.71010428
  0.70590069  0.66806837  0.63023606  0.62182888  0.61762528  0.56297861
  0.54616424  0.53355347  0.4326673   0.42426012  0.38222421  0.36120626
  0.39483498  0.37802062  0.36120626  0.38222421  0.32757753  0.33598472
  0.32337394  0.32337394  0.31496676  0.33598472  0.39063139  0.39903857
  0.4789068   0.4789068   0.63443965  0.61762528  0.64284683  0.62182888
  0.65125401  0.64284683  0.65125401  0.61342169  0.63023606  0.6092181
  0.61762528  0.58399656  0.58820015  0.55036784  0.55036784  0.52934988
  0.54616424  0.51253552  0.51253552  0.53775706  0.52934988  0.52934988
  0.54616424  0.55036784  0.55036784  0.55036784  0.52514629  0.51673911
  0.56297861  0.55036784  0.61762528  0.61342169  0.

ValueError: cannot convert float NaN to integer